In [1]:
import numpy as np
import pandas as pd


In [2]:
books = pd.read_csv('books_data/books.csv', sep=";", error_bad_lines=False, encoding="latin-1")


C:\Users\vasud\AppData\Local\Temp\ipykernel_8776\877512425.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv('books_data/books.csv', sep=";", error_bad_lines=False, encoding="latin-1")
Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields,

In [3]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-L']]

In [4]:
books.rename(columns={'Book-Title':'title','Book-Author':'author','Year-Of-Publication':'year','Image-URL-L':'url'},inplace=True)

In [5]:
users = pd.read_csv('books_data/users.csv', sep=";", error_bad_lines=False, encoding="latin-1")
ratings = pd.read_csv('books_data/ratings.csv', sep=";", error_bad_lines=False, encoding="latin-1")
ratings.rename(columns={"User-ID":"user_id","Book-Rating":"rating"},inplace=True)

C:\Users\vasud\AppData\Local\Temp\ipykernel_8776\4255861803.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  users = pd.read_csv('books_data/users.csv', sep=";", error_bad_lines=False, encoding="latin-1")


C:\Users\vasud\AppData\Local\Temp\ipykernel_8776\4255861803.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ratings = pd.read_csv('books_data/ratings.csv', sep=";", error_bad_lines=False, encoding="latin-1")


In [6]:
x=ratings['user_id'].value_counts() >200
y=x[x].index
ratings = ratings[ratings['user_id'].isin(y)]
# ratings.head(2)

In [7]:
ratings_with_books=ratings.merge(books, on="ISBN")

In [8]:
num_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [9]:
num_rating.rename(columns={"rating":"no_of_ratings"},inplace=True)

In [10]:
final_rating=ratings_with_books.merge(num_rating,on="title")

In [11]:
final_rating.head(5)

,user_id,ISBN,rating,title,author,year,Publisher,url,no_of_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [12]:
final_rating.drop_duplicates(['title','user_id'],inplace=True)

In [13]:
final_rating = final_rating[final_rating['no_of_ratings']>=50]

In [14]:
book_pivot=final_rating.pivot_table(columns="user_id",index="title",values='rating')

In [15]:
book_pivot.fillna(0, inplace=True)

In [16]:
from scipy.sparse import csr_matrix

In [17]:
book_sparse = csr_matrix(book_pivot)
book_sparse

<742x888 sparse matrix of type '<class 'numpy.float64'>'
	with 14942 stored elements in Compressed Sparse Row format>

In [18]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm="brute")

In [19]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [20]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors=10)

In [21]:
distance

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616, 77.3369252 , 77.59510294, 77.60154638, 77.65307463]])

In [22]:
suggestion

array([[237, 240, 238, 241, 184, 536, 291, 675, 686, 689]], dtype=int64)

In [23]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall', 'Jacob Have I Loved', 'The Witness',
       'Tom Clancy's Op-Center (Tom Clancy's Op Center (Paperback))', 'Toxin'],
      dtype='object', name='title')


In [24]:
book_names = book_pivot.index


In [25]:
import pickle
pickle.dump(model, open('artifacts/model.pkl','wb'))
pickle.dump(book_names, open('artifacts/book_names.pkl','wb'))
pickle.dump(final_rating, open('artifacts/final_ratings.pkl','wb'))
pickle.dump(book_pivot, open('artifacts/book_pivot.pkl','wb'))